In [2]:
import pandas as pd
%matplotlib inline
from functools import reduce

# Data Prep

## Metadata

In [28]:
DATA_PATH='/data/workspace_files/iqvia_data/'
# DATA_PATH='/Users/a206602774/Downloads/OneDrive_1_4-14-2021/'


files_claims = ['claims_2015.dat','claims_2016.dat', 'claims_2017.dat', 'claims_2018.dat', 'claims_2019.dat']
files_lookup = {
    'enroll':{'file':'enroll_synth.dat', 'key':'pat_id'},
    'enroll2':{'file':'enroll2.dat', 'key':'pat_id'},
    'dx_lookup':{'file':'pp_dx_lookup.dat', 'key':'dx_cd'},
    'pos_lookup':{'file':'pp_pos_lookup.dat', 'key':'place_of_svc_cd'},
    'pr_lookup':{'file':'pp_pr_lookup.dat', 'key':'procedure_cd'},
    'rev_lookup':{'file':'pp_rev_lookup.dat', 'key':'rev_cd'},
    'rx_lookup':{'file':'pp_rx_lookup.dat', 'key':'ndc'}}
selected_columns = ['pat_id', 'rectype', 'pos', 'conf_num', 'ndc', 'formulary', 'quan', 'proc_cde'
                   , 'from_dt', 'to_dt', 'diag_admit', 'diag1', 'diag2', 'diag3', 'icdprc1'
                   , 'icdprc2', 'icdprc3', 'paid', 'copay', 'dispense_fee', 'bill_spec'
                   , 'prscbr_spec', 'pmt_st_cd', 'paid_dt']
group_by_columns = ['pat_id', 'from_dt', 'to_dt', 'rectype', 'conf_num', 'icdprc1'
                    ,'diag_admit', 'diag1', 'proc_cde', 'bill_spec', 'pos', 'ndc', 'quan'
                    ,'formulary', 'paid_dt', 'pmt_st_cd']
date_columns = ['from_dt', 'to_dt', 'paid_dt']
cat_features = ['rectype', 'icdprc1', 'diag_admit', 'diag1', 'proc_cde', 'bill_spec', 'pos', 'ndc', 'formulary']
numerical_columns = ['quan', 'paid', 'copay', 'dispense_fee']
target_columns = ['paid', 'copay', 'dispense_fee']
e_selected_columns = ['der_sex', 'der_yob', 'pat_id', 'pat_state']
e2_selected_columns = ['pat_id', 'mh_cd']

claims_dtype = {}
for column in selected_columns:
    if column in date_columns:
        claims_dtype[column] = 'object'
    elif column in numerical_columns:
        claims_dtype[column] = 'float64'
    else:
        claims_dtype[column] = 'str'

## Data Load

In [29]:
def custom_fill_na(df):
    for col in df:
        #get dtype for column
        dt = df[col].dtype 
        #check if it is a number
        if dt == int or dt == float:
            df[col].fillna(0, inplace=True)
        else:
            df[col].fillna("-", inplace=True)
        
def load_data(files):
    # Load Claims_15 data (revisit later)
    data = pd.DataFrame()
    for file in files:
        print(file)
        data = data.append(pd.read_table(DATA_PATH+file, delimiter="|"
                                         , usecols = selected_columns
                                         , dtype = claims_dtype)) 
    custom_fill_na(data)        
    # Create aggregate dataset for training
    data = data.groupby(group_by_columns).sum().reset_index()
    # Data type cleanup and calculate derived columns     
    for column in date_columns: 
        data[column] = pd.to_datetime(data[column])

    return data

In [30]:
# Only loading 2015 and 2016 data
combined_data = load_data(files_claims[0:2])

claims_2015.dat
claims_2016.dat


In [31]:
dx_lookup = pd.read_table(DATA_PATH+files_lookup['dx_lookup']['file'],delimiter="|")
pos_lookup = pd.read_table(DATA_PATH+files_lookup['pos_lookup']['file'],delimiter="|")
pr_lookup = pd.read_table(DATA_PATH+files_lookup['pr_lookup']['file'],delimiter="|")
rev_lookup = pd.read_table(DATA_PATH+files_lookup['rev_lookup']['file'],delimiter="|")

''' 
We only want the values that are available in the lookup
'''
dx_key = files_lookup['dx_lookup']['key']
pos_key = files_lookup['pos_lookup']['key'] 
pr_key = files_lookup['pr_lookup']['key'] 
# rev_key = files_lookup['rev_lookup']['key'] 

combined_data = combined_data[(combined_data['diag1'].isin(dx_lookup[dx_key])) | (combined_data['diag1']=="-")]
combined_data = combined_data[(combined_data['diag_admit'].isin(dx_lookup[dx_key])) | (combined_data['diag_admit']=="-")]
# print(combined_data[(combined_data['pos'].isin(pos_lookup[pos_key])) | (combined_data['pos']=="")].shape)
combined_data = combined_data[(combined_data['proc_cde'].isin(pr_lookup[pr_key])) | (combined_data['proc_cde']=="-")]

enroll = pd.read_table(DATA_PATH+files_lookup['enroll']['file'],delimiter="|", usecols = e_selected_columns)

data_frames = [combined_data, enroll]
combined_data = reduce(lambda  left,right: pd.merge(left,right,on=['pat_id'],how='inner'), data_frames)
# Derived columns 
combined_data['quarter'] = combined_data['to_dt'].dt.to_period("Q")
combined_data['duration'] = combined_data['to_dt'] - combined_data['from_dt']
duration_bins= [pd.Timedelta(days = 0),
                pd.Timedelta(days = 7),
                pd.Timedelta(days = 15),
                pd.Timedelta(days = 30),
                pd.Timedelta(days = 60),
                pd.Timedelta(days = 5000)]
duration_groups = ['<7', '8-15', '16-30', '31-60', '60>']
combined_data['duration'] = pd.cut(combined_data['duration'], bins=duration_bins, labels=duration_groups, right=False)


combined_data['pat_age'] = combined_data['to_dt'].dt.year - combined_data['der_yob']
pat_age_bins= [0,10,20,30,40,50,60,70,80,150]
pat_age_groups = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81 >']
combined_data['age_group'] = pd.cut(combined_data['pat_age'], bins=pat_age_bins, labels=pat_age_groups, right=False)

combined_data

,pat_id,from_dt,to_dt,rectype,conf_num,icdprc1,diag_admit,diag1,proc_cde,bill_spec,...,paid,copay,dispense_fee,der_sex,der_yob,pat_state,quarter,duration,pat_age,age_group
0,0309AAAAAAAABEUP,2015-01-02,2015-01-02,M,-,-,-,7391,98941,CHIRO,...,0.00,0.0,0.0,F,1996.0,MN,2015Q1,<7,19.0,11-20
1,0309AAAAAAAABEUP,2015-01-05,2015-01-05,M,-,-,-,7391,98941,CHIRO,...,0.00,0.0,0.0,F,1996.0,MN,2015Q1,<7,19.0,11-20
2,0309AAAAAAAABEUP,2015-01-20,2015-01-20,M,-,-,-,4770,95117,ALLERGY,...,0.00,0.0,0.0,F,1996.0,MN,2015Q1,<7,19.0,11-20
3,0309AAAAAAAABEUP,2015-02-10,2015-02-10,M,-,-,-,4770,95117,ALLERGY,...,0.00,0.0,0.0,F,1996.0,MN,2015Q1,<7,19.0,11-20
4,0309AAAAAAAABEUP,2015-03-03,2015-03-03,M,-,-,-,4770,95117,ALLERGY,...,0.00,0.0,0.0,F,1996.0,MN,2015Q1,<7,19.0,11-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385435,s153AAAAAQWAQSGZ,2016-12-28,2016-12-28,S,-,-,-,S8331XA,20610,GP_FP,...,181.22,0.0,0.0,M,1963.0,WA,2016Q4,<7,53.0,51-60
1385436,s153AAAAAQWAQSGZ,2016-12-29,2016-12-29,-,-,-,-,-,-,-,...,7.00,0.0,0.0,M,1963.0,WA,2016Q4,<7,53.0,51-60
1385437,s153AAAAAQWAQSGZ,2016-12-29,2016-12-29,-,-,-,-,-,-,-,...,13.42,0.0,0.0,M,1963.0,WA,2016Q4,<7,53.0,51-60
1385438,s153AAAAAQWAQSGZ,2016-12-29,2016-12-29,P,-,-,-,-,-,-,...,3.79,0.0,0.0,M,1963.0,WA,2016Q4,<7,53.0,51-60


In [32]:
column_order = ['pat_id', 'quarter','from_dt', 'to_dt', 'duration', 'paid_dt', 'rectype', 'conf_num', 'icdprc1',
       'diag_admit', 'diag1', 'proc_cde', 'ndc', 'bill_spec', 'pos','der_sex', 'der_yob', 'pat_state',  'pat_age',
       'age_group', 'quan', 'formulary',  'pmt_st_cd', 'paid', 'copay', 'dispense_fee']

set(column_order) == set(combined_data.columns)

True

In [33]:
combined_data = combined_data[column_order]

In [34]:
combined_data

,pat_id,quarter,from_dt,to_dt,duration,paid_dt,rectype,conf_num,icdprc1,diag_admit,...,der_yob,pat_state,pat_age,age_group,quan,formulary,pmt_st_cd,paid,copay,dispense_fee
0,0309AAAAAAAABEUP,2015Q1,2015-01-02,2015-01-02,<7,2015-02-01,M,-,-,-,...,1996.0,MN,19.0,11-20,0.0,-,P,0.00,0.0,0.0
1,0309AAAAAAAABEUP,2015Q1,2015-01-05,2015-01-05,<7,2015-02-04,M,-,-,-,...,1996.0,MN,19.0,11-20,0.0,-,P,0.00,0.0,0.0
2,0309AAAAAAAABEUP,2015Q1,2015-01-20,2015-01-20,<7,2015-02-19,M,-,-,-,...,1996.0,MN,19.0,11-20,0.0,-,P,0.00,0.0,0.0
3,0309AAAAAAAABEUP,2015Q1,2015-02-10,2015-02-10,<7,2015-03-12,M,-,-,-,...,1996.0,MN,19.0,11-20,0.0,-,P,0.00,0.0,0.0
4,0309AAAAAAAABEUP,2015Q1,2015-03-03,2015-03-03,<7,2015-04-02,M,-,-,-,...,1996.0,MN,19.0,11-20,0.0,-,P,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385435,s153AAAAAQWAQSGZ,2016Q4,2016-12-28,2016-12-28,<7,2017-01-27,S,-,-,-,...,1963.0,WA,53.0,51-60,0.0,-,P,181.22,0.0,0.0
1385436,s153AAAAAQWAQSGZ,2016Q4,2016-12-29,2016-12-29,<7,2017-01-03,-,-,-,-,...,1963.0,WA,53.0,51-60,30.0,Y,P,7.00,0.0,0.0
1385437,s153AAAAAQWAQSGZ,2016Q4,2016-12-29,2016-12-29,<7,2017-01-03,-,-,-,-,...,1963.0,WA,53.0,51-60,60.0,Y,P,13.42,0.0,0.0
1385438,s153AAAAAQWAQSGZ,2016Q4,2016-12-29,2016-12-29,<7,2017-01-03,P,-,-,-,...,1963.0,WA,53.0,51-60,7.0,Y,P,3.79,0.0,0.0


In [35]:
combined_data.der_sex.unique()

array(['F', 'M', 'U'], dtype=object)

In [36]:
combined_data.shape

(1385440, 26)

In [37]:
combined_data.to_csv(DATA_PATH+'clean/model_input.csv', sep='|')

In [38]:
combined_data.dtypes

pat_id                  object
quarter          period[Q-DEC]
from_dt         datetime64[ns]
to_dt           datetime64[ns]
duration              category
paid_dt         datetime64[ns]
rectype                 object
conf_num                object
icdprc1                 object
diag_admit              object
diag1                   object
proc_cde                object
ndc                     object
bill_spec               object
pos                     object
der_sex                 object
der_yob                float64
pat_state               object
pat_age                float64
age_group             category
quan                   float64
formulary               object
pmt_st_cd               object
paid                   float64
copay                  float64
dispense_fee           float64
dtype: object